In [10]:
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from sklearn.preprocessing import OneHotEncoder
from collections import Counter

import segmentation_models_pytorch as smp
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.cuda.amp import autocast, GradScaler

from tqdm import tqdm


In [3]:
import torch.backends.cudnn as cudnn
import os
#cudnn.benchmark = True


In [11]:
# Load data
df_train = pd.read_pickle(r"C:\Users\gnvca\OneDrive\Desktop\JP\Model_Train.pkl")
df_val = pd.read_pickle(r"C:\Users\gnvca\OneDrive\Desktop\JP\Model_Val.pkl")
df_train = df_train[df_train["img_origin"] == "D"].reset_index(drop=True)
df_val = df_val[df_val["img_origin"] == "D"].reset_index(drop=True)

This model is using a different strategy:
- The metadata was encoded onto the images via one hot encoding 
- Based on the 2 classes and 2 origins, the class balancing was attempted for the 4 classes during the albumentations step (although officially there are only 2 classes still, solar and boiler)


In [6]:
# Function to create multi-class mask
def create_multi_class_mask(image_size, polygons_boil, polygons_pan):
    mask = np.full(image_size, 1, dtype=np.uint8)  # Default background is Photovoltaic (1)
    
    # Draw boiler panels (0)
    for polygon in polygons_boil:
        cv2.fillPoly(mask, np.array([polygon], dtype=np.int32), 0)

    return mask


# One-hot encode metadata
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
metadata_encoded = encoder.fit_transform(df_train[['img_placement', 'img_origin']])

# Define transformation pipelines
albumentations_transform = A.Compose([
    A.Resize(512, 512),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.GaussianBlur(p=0.2),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

# Dataset class
class SolarPanelDataset(Dataset):
    def __init__(self, metadata_df, image_dir, transform=None, mask_size=(512, 512), balance=False):
        self.metadata = metadata_df
        self.image_dir = image_dir
        self.transform = transform
        self.mask_size = mask_size
        self.balance = balance
        
        # One-hot encode metadata
        self.encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        self.encoded_metadata = self.encoder.fit_transform(self.metadata[['img_placement', 'img_origin']])
        
        # Create class labels for balancing
        self.class_labels = self.metadata.apply(lambda row: f"{row['img_origin']}_{'solar' if row['polygons_pan'] else 'boiler'}", axis=1)
        
        # Compute class weights for balancing
        if balance:
            class_counts = Counter(self.class_labels)
            self.weights = [1.0 / class_counts[label] for label in self.class_labels]
        else:
            self.weights = None

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        row = self.metadata.iloc[idx]
        img_path = f"{self.image_dir}/{row['img_id']}.jpg"
        image = np.array(Image.open(img_path).convert("RGB"))

        # Create the mask
        mask = create_multi_class_mask(image.shape[:2], row['polygons_boil'], row['polygons_pan'])
        mask = np.array(mask, dtype=np.uint8)

        # Apply transformations
        augmented = self.transform(image=image, mask=mask)
        image, mask = augmented["image"], augmented["mask"]

        # Convert mask to long tensor
        if isinstance(mask, np.ndarray):  # Convert only if it's still a NumPy array
            mask = torch.from_numpy(mask).long()
        else:
            mask = mask.long()  # If it's already a tensor, just ensure dtype

        # Get one-hot encoded metadata
        metadata_vector = torch.tensor(self.encoded_metadata[idx], dtype=torch.float32)

        return image, mask, metadata_vector  # Return metadata as additional input

# Define image directory
image_dir = r"C:\Users\gnvca\OneDrive\Desktop\JP\images"

# Create train dataset with class balancing
train_dataset = SolarPanelDataset(df_train, image_dir, transform=albumentations_transform, balance=True)
val_dataset = SolarPanelDataset(df_val, image_dir, transform=A.Compose([
    A.Resize(512, 512),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
]))

# Create Weighted Sampler for class balancing
if train_dataset.weights:
    sampler = WeightedRandomSampler(weights=train_dataset.weights, num_samples=len(train_dataset), replacement=True)
else:
    sampler = None

# Create DataLoaders
batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler if sampler else None, shuffle=sampler is None, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [7]:
# Load DeepLabV3+ with EfficientNet-B4 backbone
model = smp.DeepLabV3Plus(
    encoder_name="efficientnet-b4",  # EfficientNet-B4 as the encoder
    encoder_weights="imagenet",  # Pretrained weights
    in_channels=3,  # RGB images
    classes=2  # Boiler (0), Photovoltaic (1)
)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Add dropout before the classifier correctly
model.segmentation_head = nn.Sequential(
    nn.Dropout(0.3),  # 30% dropout
    model.segmentation_head
)

# Define loss function (CrossEntropy + Dice Loss for better performance)
criterion = nn.CrossEntropyLoss()
dice_loss = smp.losses.DiceLoss(mode='multiclass')

# Adam optimizer with weight decay
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

# Learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

# Mixed precision scaler for faster GPU training
scaler = torch.amp.GradScaler(device="cuda")

c:\Users\gnvca\anaconda3\envs\caa01\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [ ]:
# Function to calculate IoU
def iou_score(preds, labels, num_classes=2):
    """Compute IoU (Intersection over Union) for multi-class segmentation."""
    preds = torch.argmax(preds, dim=1)  # Convert logits to class predictions
    iou = []

    for cls in range(num_classes):
        intersection = ((preds == cls) & (labels == cls)).sum().item()
        union = ((preds == cls) | (labels == cls)).sum().item()
        if union == 0:
            iou.append(float('nan'))
        else:
            iou.append(intersection / union)

    return np.nanmean(iou)  # Ignore NaNs if a class is missing in batch


# 🔹 Model Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = smp.DeepLabV3Plus(
    encoder_name="efficientnet-b4",
    encoder_weights="imagenet",
    in_channels=3,
    classes=2
).to(device)

# 🔹 Add Dropout Correctly
model.segmentation_head = nn.Sequential(
    nn.Dropout(0.3),
    model.segmentation_head
)

# 🔹 Loss Functions (CrossEntropy + Dice Loss)
criterion = nn.CrossEntropyLoss()
dice_loss = smp.losses.DiceLoss(mode='multiclass')

# 🔹 Optimizer & LR Scheduler
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

# 🔹 Mixed Precision (Speeds up Training)
scaler = torch.amp.GradScaler(device="cuda")

history = {
    "epoch": [],
    "train_loss": [],
    "train_iou": [],
    "val_loss": [],
    "val_iou": []
}

output_dir = r"C:\Users\gnvca\OneDrive\Desktop\JP\\"

# Training Hyperparameters
num_epochs = 50
best_val_loss = float("inf")
accumulation_steps = 4  # Simulates larger batch size

# 🔹 Training Loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    total_iou = 0.0
    num_batches = 0

    optimizer.zero_grad()  # Initialize gradients before accumulation

    for i, (images, masks, _) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} Training", leave=True, dynamic_ncols=True)):
        images, masks = images.to(device), masks.to(device)

        with torch.amp.autocast(device_type="cuda", dtype=torch.float16):  # Enables mixed precision
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, masks) + dice_loss(outputs, masks)  # Combined loss
        
        scaler.scale(loss).backward()  # Accumulate gradients

        # 🔹 Only update every `accumulation_steps`
        if (i + 1) % accumulation_steps == 0 or (i + 1) == len(train_loader):
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()  # Reset gradients

        running_loss += loss.item()
        total_iou += iou_score(outputs, masks, num_classes=2)
        num_batches += 1
        
        #tqdm.write(f"Batch {i+1}/{len(train_loader)} - Loss: {loss.item():.4f}")

    avg_train_loss = running_loss / num_batches
    avg_train_iou = total_iou / num_batches

    # 🔹 Validation Loop
    model.eval()
    val_loss = 0.0
    val_iou = 0.0
    num_batches = 0

    with torch.no_grad():
        for images, masks, _ in tqdm(val_loader, desc="Validation"):
            images, masks = images.to(device), masks.to(device)

            with torch.amp.autocast(device_type="cuda", dtype=torch.float16):  # Use mixed precision in inference
                outputs = model(images)
                loss = criterion(outputs, masks) + dice_loss(outputs, masks)

            val_loss += loss.item()
            val_iou += iou_score(outputs, masks, num_classes=2)
            num_batches += 1

    avg_val_loss = val_loss / num_batches
    avg_val_iou = val_iou / num_batches

    # 🔥 Save Best Model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), f"bm_effdeepl_D_epoch{epoch}.pth")
        print("🔥 Best Model Saved!")

    # 🔹 Logging
    print(f"\n🔹 Epoch {epoch+1}/{num_epochs}")
    print(f"   📉 Train Loss: {avg_train_loss:.4f} | 🏆 Train IoU: {avg_train_iou:.4f}")
    print(f"   📉 Val Loss: {avg_val_loss:.4f} | 🏆 Val IoU: {avg_val_iou:.4f}")

    history["epoch"].append(epoch + 1)
    history["train_loss"].append(avg_train_loss)
    history["train_iou"].append(avg_train_iou)
    history["val_loss"].append(avg_val_loss)
    history["val_iou"].append(avg_val_iou)

    history_df = pd.DataFrame(history)
    history_path = os.path.join(output_dir, "training_history_02_D.csv")
    history_df.to_csv(history_path, index=False)
    print(f"📊 Training history saved to: {history_path}")
    
    # 🔹 Adjust LR based on Validation Loss
    scheduler.step(avg_val_loss)




Validation: 100%|██████████| 123/123 [01:05<00:00,  1.87it/s]


🔥 Best Model Saved!

🔹 Epoch 1/50
   📉 Train Loss: 0.6865 | 🏆 Train IoU: 0.4950
   📉 Val Loss: 0.1182 | 🏆 Val IoU: 0.9593
📊 Training history saved to: C:\Users\gnvca\OneDrive\Desktop\JP\\training_history_02_D.csv


Validation: 100%|██████████| 123/123 [01:07<00:00,  1.81it/s]


🔥 Best Model Saved!

🔹 Epoch 2/50
   📉 Train Loss: 0.5142 | 🏆 Train IoU: 0.4999
   📉 Val Loss: 0.0644 | 🏆 Val IoU: 0.9593
📊 Training history saved to: C:\Users\gnvca\OneDrive\Desktop\JP\\training_history_02_D.csv


Validation: 100%|██████████| 123/123 [01:08<00:00,  1.81it/s]


...

Validation: 100%|██████████| 123/123 [00:57<00:00,  2.14it/s]



🔹 Epoch 14/50
   📉 Train Loss: 0.1995 | 🏆 Train IoU: 0.7389
   📉 Val Loss: 0.0407 | 🏆 Val IoU: 0.9553
📊 Training history saved to: C:\Users\gnvca\OneDrive\Desktop\JP\\training_history_02_D.csv


Validation: 100%|██████████| 123/123 [00:56<00:00,  2.18it/s]



🔹 Epoch 15/50
   📉 Train Loss: 0.1873 | 🏆 Train IoU: 0.7411
   📉 Val Loss: 0.0406 | 🏆 Val IoU: 0.9593
📊 Training history saved to: C:\Users\gnvca\OneDrive\Desktop\JP\\training_history_02_D.csv


Epoch 16/50 Training:  16%|█▋        | 82/500 [00:49<04:17,  1.62it/s]

This model is very promising, with a substantial IoU in comparison to the combined approach.

In [12]:
# Load data
df_train = pd.read_pickle(r"C:\Users\gnvca\OneDrive\Desktop\JP\Model_Train.pkl")
df_val = pd.read_pickle(r"C:\Users\gnvca\OneDrive\Desktop\JP\Model_Val.pkl")
df_train = df_train[df_train["img_origin"] == "S"].reset_index(drop=True)
df_val = df_val[df_val["img_origin"] == "S"].reset_index(drop=True)

In [13]:
# Convert stringified lists to actual lists if needed
import ast

def safe_eval(val):
    if isinstance(val, str):
        return ast.literal_eval(val)
    return val

df_train["polygons_boil"] = df_train["polygons_boil"].apply(safe_eval)
df_train["polygons_pan"] = df_train["polygons_pan"].apply(safe_eval)

# Count the number of images with more than 1 polygon in either field
def has_multiple_panels(polygons_boil, polygons_pan):
    return len(polygons_boil) > 1 or len(polygons_pan) > 1

df_train["has_multiple_panels"] = df_train.apply(
    lambda row: has_multiple_panels(row["polygons_boil"], row["polygons_pan"]),
    axis=1
)

# Group and count
panel_counts = df_train["has_multiple_panels"].value_counts()

print("✅ Grouped count of images with multiple panels:")
print(panel_counts)


✅ Grouped count of images with multiple panels:
has_multiple_panels
False    296
True     145
Name: count, dtype: int64


In [14]:
import torch
import torch.nn as nn
import segmentation_models_pytorch as smp

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Rebuild model architecture exactly as before
model = smp.DeepLabV3Plus(
    encoder_name="efficientnet-b4",
    encoder_weights=None,  # Don't load ImageNet again
    in_channels=3,
    classes=2
)

# Add dropout before loading weights (must match original architecture)
model.segmentation_head = nn.Sequential(
    nn.Dropout(0.3),
    model.segmentation_head
)

# Load weights (strict=True now that it matches)
checkpoint_path = r"C:\Users\gnvca\OneDrive\Desktop\JP\model02D\bm_effdeepl_D_epoch10.pth"
model.load_state_dict(torch.load(checkpoint_path, map_location=device), strict=True)

# Move to device
model.to(device)

print("✅ Model with Dropout segmentation head loaded successfully.")


✅ Model with Dropout segmentation head loaded successfully.


In [18]:
def convert_batchnorm_to_groupnorm(model, num_groups=32):
    for name, module in model.named_children():
        if isinstance(module, torch.nn.BatchNorm2d):
            setattr(model, name, torch.nn.GroupNorm(num_groups, module.num_features))
        else:
            convert_batchnorm_to_groupnorm(module, num_groups)

# Function to create multi-class mask
def create_multi_class_mask(image_size, polygons_boil, polygons_pan):
    mask = np.full(image_size, 1, dtype=np.uint8)  # Default background is Photovoltaic (1)
    
    # Draw boiler panels (0)
    for polygon in polygons_boil:
        cv2.fillPoly(mask, np.array([polygon], dtype=np.int32), 0)

    return mask


# One-hot encode metadata
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
metadata_encoded = encoder.fit_transform(df_train[['img_placement', 'img_origin']])

# Define transformation pipelines
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.Resize(512, 512),  # ✅ Ensure final size is large enough
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])


# Define transformation for training
albumentations_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.Resize(512, 512),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# Dataset class
class SolarPanelDataset(Dataset):
    def __init__(self, metadata_df, image_dir, transform=None, mask_size=(512, 512), balance=False):
        self.metadata = metadata_df
        self.image_dir = image_dir
        self.transform = transform
        self.mask_size = mask_size
        self.balance = balance
        
        # One-hot encode metadata
        self.encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        self.encoded_metadata = self.encoder.fit_transform(self.metadata[['img_placement', 'img_origin']])
        
        # Create class labels for balancing
        self.class_labels = self.metadata.apply(lambda row: f"{row['img_origin']}_{'solar' if row['polygons_pan'] else 'boiler'}", axis=1)
        
        # Compute class weights for balancing
        if balance:
            class_counts = Counter(self.class_labels)
            self.weights = [1.0 / class_counts[label] for label in self.class_labels]
        else:
            self.weights = None

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        row = self.metadata.iloc[idx]
        img_path = f"{self.image_dir}/{row['img_id']}.jpg"
        image = np.array(Image.open(img_path).convert("RGB"))

        # Create the mask
        mask = create_multi_class_mask(image.shape[:2], row['polygons_boil'], row['polygons_pan'])
        mask = np.array(mask, dtype=np.uint8)

        # Apply transformations
        augmented = self.transform(image=image, mask=mask)
        image, mask = augmented["image"], augmented["mask"]

        # Convert mask to long tensor
        if isinstance(mask, np.ndarray):  # Convert only if it's still a NumPy array
            mask = torch.from_numpy(mask).long()
        else:
            mask = mask.long()  # If it's already a tensor, just ensure dtype

        # Get one-hot encoded metadata
        metadata_vector = torch.tensor(self.encoded_metadata[idx], dtype=torch.float32)

        return image, mask, metadata_vector  # Return metadata as additional input

# Define image directory
image_dir = r"C:\Users\gnvca\OneDrive\Desktop\JP\images"

# Create train dataset with class balancing
train_dataset = SolarPanelDataset(df_train, image_dir, transform=albumentations_transform, balance=True)
val_dataset = SolarPanelDataset(df_val, image_dir, transform=A.Compose([
    A.Resize(512, 512),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
]))

# Create Weighted Sampler for class balancing
if train_dataset.weights:
    sampler = WeightedRandomSampler(weights=train_dataset.weights, num_samples=len(train_dataset), replacement=True)
else:
    sampler = None

# Create DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler if sampler else None, shuffle=sampler is None, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)



In [20]:
import os
# Function to calculate IoU
def iou_score(preds, labels, num_classes=2):
    """Compute IoU (Intersection over Union) for multi-class segmentation."""
    preds = torch.argmax(preds, dim=1)  # Convert logits to class predictions
    iou = []

    for cls in range(num_classes):
        intersection = ((preds == cls) & (labels == cls)).sum().item()
        union = ((preds == cls) | (labels == cls)).sum().item()
        if union == 0:
            iou.append(float('nan'))
        else:
            iou.append(intersection / union)

    return np.nanmean(iou)  # Ignore NaNs if a class is missing in batch


# 🔹 Loss Functions (CrossEntropy + Dice Loss)
criterion = nn.CrossEntropyLoss()
dice_loss = smp.losses.DiceLoss(mode='multiclass')

# 🔹 Optimizer & LR Scheduler
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

# 🔹 Mixed Precision (Speeds up Training)
scaler = torch.amp.GradScaler(device="cuda")

history = {
    "epoch": [],
    "train_loss": [],
    "train_iou": [],
    "val_loss": [],
    "val_iou": []
}

output_dir = r"C:\Users\gnvca\OneDrive\Desktop\JP\\"

# Training Hyperparameters
num_epochs = 50
best_val_loss = float("inf")
accumulation_steps = 4  # Simulates larger batch size

# 🔹 Training Loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    total_iou = 0.0
    num_batches = 0

    optimizer.zero_grad()  # Initialize gradients before accumulation

    for i, (images, masks, _) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} Training", leave=True, dynamic_ncols=True)):
        images, masks = images.to(device), masks.to(device)

        with torch.amp.autocast(device_type="cuda", dtype=torch.float16):  # Enables mixed precision
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, masks) + dice_loss(outputs, masks)  # Combined loss
        
        scaler.scale(loss).backward()  # Accumulate gradients

        # 🔹 Only update every `accumulation_steps`
        if (i + 1) % accumulation_steps == 0 or (i + 1) == len(train_loader):
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()  # Reset gradients

        running_loss += loss.item()
        total_iou += iou_score(outputs, masks, num_classes=2)
        num_batches += 1
        
        #tqdm.write(f"Batch {i+1}/{len(train_loader)} - Loss: {loss.item():.4f}")

    avg_train_loss = running_loss / num_batches
    avg_train_iou = total_iou / num_batches

    # 🔹 Validation Loop
    model.eval()
    val_loss = 0.0
    val_iou = 0.0
    num_batches = 0

    with torch.no_grad():
        for images, masks, _ in tqdm(val_loader, desc="Validation"):
            images, masks = images.to(device), masks.to(device)

            with torch.amp.autocast(device_type="cuda", dtype=torch.float16):  # Use mixed precision in inference
                outputs = model(images)
                loss = criterion(outputs, masks) + dice_loss(outputs, masks)

            val_loss += loss.item()
            val_iou += iou_score(outputs, masks, num_classes=2)
            num_batches += 1

    avg_val_loss = val_loss / num_batches
    avg_val_iou = val_iou / num_batches

    # 🔥 Save Best Model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), f"bm_effdeepl_D_epoch{epoch}.pth")
        print("🔥 Best Model Saved!")

    # 🔹 Logging
    print(f"\n🔹 Epoch {epoch+1}/{num_epochs}")
    print(f"   📉 Train Loss: {avg_train_loss:.4f} | 🏆 Train IoU: {avg_train_iou:.4f}")
    print(f"   📉 Val Loss: {avg_val_loss:.4f} | 🏆 Val IoU: {avg_val_iou:.4f}")

    history["epoch"].append(epoch + 1)
    history["train_loss"].append(avg_train_loss)
    history["train_iou"].append(avg_train_iou)
    history["val_loss"].append(avg_val_loss)
    history["val_iou"].append(avg_val_iou)

    history_df = pd.DataFrame(history)
    history_path = os.path.join(output_dir, "training_history_02_D.csv")
    history_df.to_csv(history_path, index=False)
    print(f"📊 Training history saved to: {history_path}")
    
    # 🔹 Adjust LR based on Validation Loss
    scheduler.step(avg_val_loss)




c:\Users\gnvca\anaconda3\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
Validation: 100%|██████████| 8/8 [00:17<00:00,  2.20s/it]


🔥 Best Model Saved!

🔹 Epoch 1/50
   📉 Train Loss: 0.5280 | 🏆 Train IoU: 0.5043
   📉 Val Loss: 0.5501 | 🏆 Val IoU: 0.4972
📊 Training history saved to: C:\Users\gnvca\OneDrive\Desktop\JP\\training_history_02_D.csv


Validation: 100%|██████████| 8/8 [00:17<00:00,  2.17s/it]


🔥 Best Model Saved!

🔹 Epoch 2/50
   📉 Train Loss: 0.4961 | 🏆 Train IoU: 0.5306
   📉 Val Loss: 0.5368 | 🏆 Val IoU: 0.4980
📊 Training history saved to: C:\Users\gnvca\OneDrive\Desktop\JP\\training_history_02_D.csv


Validation: 100%|██████████| 8/8 [00:17<00:00,  2.15s/it]


...


🔹 Epoch 48/50
   📉 Train Loss: 0.3527 | 🏆 Train IoU: 0.6267
   📉 Val Loss: 0.5324 | 🏆 Val IoU: 0.5025
📊 Training history saved to: C:\Users\gnvca\OneDrive\Desktop\JP\\training_history_02_D.csv


Validation: 100%|██████████| 8/8 [00:16<00:00,  2.12s/it]



🔹 Epoch 49/50
   📉 Train Loss: 0.3777 | 🏆 Train IoU: 0.6067
   📉 Val Loss: 0.5329 | 🏆 Val IoU: 0.5023
📊 Training history saved to: C:\Users\gnvca\OneDrive\Desktop\JP\\training_history_02_D.csv


Validation: 100%|██████████| 8/8 [00:17<00:00,  2.24s/it]


🔹 Epoch 50/50
   📉 Train Loss: 0.3547 | 🏆 Train IoU: 0.6218
   📉 Val Loss: 0.5342 | 🏆 Val IoU: 0.5003
📊 Training history saved to: C:\Users\gnvca\OneDrive\Desktop\JP\\training_history_02_D.csv
